# Peak selection

From chromatin accessibility anndata (peaks called), select peaks for downstream QTL testing and other tasks.

Inputs:
- ! Temp: FOOTPRINTS_DIR/ footprints_<CT>_processed.h5ad

Outputs:

Notes:
- Source of peak selection should change to here.

## Setup

In [8]:
import os
import sys
# import json
# import gc
# from datetime import date
# import numpy as np
# import pandas as pd
# import anndata as ad
# import pyBigWig

In [21]:
# Ensure cwd is project root and that /code is in path

import os
import sys

cwd = os.path.basename(os.getcwd())

if cwd == 'footprintQTL':
    
    PROJECT_DIR = '.'


elif cwd == 'code':
    
    PROJECT_DIR = '..'


elif cwd == 'fichtner':

    PROJECT_DIR = 'projects/footprintQTL'

else:

    PROJECT_DIR = 'manual'


os.chdir(PROJECT_DIR)
sys.path.append(os.getcwd() + '/code')

In [10]:
from helpers.python.utils import create_dir, list_files_and_links, list_dirs, read_rownames

## Variables

In [11]:
# User variables

from glob_vars import \
                    MAIN_ENV, \
                    DATASET, CT_MAP_ID, \
                    ATAC_CHROM_ACCESS_METADATA_DIR, SELECT_PEAKS_TSV_DIR, \
                    CHROM_SIZES, SNP_LOCS_BED
#                       GROUPED_BIGWIG_FILES_DIR, \
#FOOTPRINTS_DIR, FOOTPRINTS_METADATA_DIR, \
#                       DATASET, CT_MAP_JSON \
#                       GENOTYPES_PROCESSED_TSV, GENOTYPE_PCS_TSV, \
#                       MATRIX_EQTL_INPUT_DIR, MATRIX_EQTL_OUTPUT_DIR, \


# cell_type = str("DL-EN")
scales = (16, 26, 51, 101)
radii = tuple(scale // 2 for scale in scales)


In [12]:
os.environ['PATH'] = f'/omics/groups/OE0540/internal_temp/users/fichtner/micromamba/envs/{MAIN_ENV}/bin:' + os.environ['PATH']
from pybedtools import BedTool

# Load data

In [13]:
snps_bed = BedTool(SNP_LOCS_BED)

## Variant-centred peaks

In [23]:
os.getcwd()

'/omics/groups/OE0540/internal_temp/users/fichtner/projects/footprintQTL'

In [26]:
from glob_vars import SELECT_PEAKS_TSV_DIR


IN_DIR = f'data/intermediate-data/datasets/{DATASET}/matrix-eqtl/chromatin-accessibility/matrix-eQTL_io_5caPCs/chromatin_accessibility/peak_ca/{CT_MAP_ID}'
in_files = list_files_and_links(IN_DIR, extension='.tsv')

for file in in_files:

    cell_type = os.path.splitext(file)[0]
    print(cell_type)

    ## Get select peaks bed
    peaks = read_rownames(os.path.join(IN_DIR, file), header=True)

    peaks_bed_list = []

    for peak in peaks:

        fields = peak.split(':')
        entry = '{} {} {} {}'.format(fields[0], str(int(fields[1]) - 1), fields[2], peak) # Indexing basis correction: 1-based fully closed -> 0-based half open
        peaks_bed_list.append(entry)

    peaks_bed = '\n'.join(peaks_bed_list)

    peaks_bed = BedTool(peaks_bed, from_string=True)

    # Save

    peak_bed_out = os.path.join(SELECT_PEAKS_TSV_DIR, cell_type, 'peaks_ca-qtls.bed')
    create_dir(peak_bed_out)
    peaks_bed.saveas(peak_bed_out)


    ## SNPs overlapping peaks

    for radius, scale in zip(radii, scales):

        snps_overlap = snps_bed.intersect(peaks_bed, wa=True)
        peaks_vc_bed = snps_overlap.slop(b=radius, g=CHROM_SIZES)

        # Save

        peaks_vc_bed_out = os.path.join(SELECT_PEAKS_TSV_DIR, cell_type, f'peaks_ca-qtls_variant-centred_{scale}bp.bed')
        create_dir(peaks_vc_bed_out)
        peaks_vc_bed.saveas(peaks_vc_bed_out)




Neural-progenitors
Midbrain-EN
Glia
DL-EN
UL-EN
